In [1]:
from __future__ import division
import tensor_calculation as t
from i import Ima
from collections import Counter
import random
from math import exp
from math import log

In [2]:
def gene_random_tensor(struct, (a,b)): 
    if type(struct) != list:
        "struct must be a tensor"
    elif len(struct) == 1:
        return [random.uniform(a,b) for i in range(struct[0])]
    else:
        return[gene_random_tensor(struct[1:], (a,b)) for i in range(struct[0])]

In [3]:
def gene_blank_tensor(struct):
    if type(struct) != list:
        "struct must be a tensor"
    elif len(struct) == 1:
        return [0 for i in range(struct[0])]
    else:
        return[gene_blank_tensor(struct[1:]) for i in range(struct[0])]

In [4]:
def sig (list_):
    if len(list_) ==1:
        return 1
    counter = 0
    for i in range(len(list_) -1):
        for j in range(i+1):
            if list_[i+1]<list_[j]:
                counter += 1
    return (-1)**counter       

In [5]:
def gk_delta(lenth, dim):                      #generalized Kronecker delta, we want up index and down index
    struct = [dim for j in range(2*lenth)]           
    bl = [0 for k in range(dim**(2*lenth))]
    for i in range(len(bl)):
        whole_cord = t.n_to_cord(struct, i)
        u_cord=whole_cord[0:lenth]
        d_cord=whole_cord[lenth:]
        if max(Counter(u_cord).values())>1 or max(Counter(d_cord).values() )>1\
            or  Counter(u_cord).keys() != Counter(d_cord).keys(): 
            continue  
        bl[i] = sig(u_cord)*sig(d_cord)
    return t.list_to_tensor(bl, struct)

In [6]:
def matrix_adj(A ,i,j):
    if len(A) == 1:
        return A
    b=  A[0:i]+A[i+1:]
    return [ be[0:j]+be[j+1:] for be in b]

In [7]:
def det(a):
    if len(a) ==1:
        return a[0][0]
    return sum([a[0][i]*((-1)**i )*det( matrix_adj(a ,0,i) )  for i in range(len(a)) ] )

In [8]:
def matrix_inver(a):
    if len(a) !=len(a[0]):
        return "not a squire matrix"
    if len(a) ==1 and len(a[0]) ==1:
        return[[1/a[0][0]]]
    b = []
    for i in range(len(a)):
        b.append([t.list_nu(det(matrix_adj(a, i,j) ), (-1)**(i +j)   )  for j in range(len(a[0])) ] )
    return t.list_nu(t.transp(b, [1,0]), 1/det(a)) 

In [9]:
def colu_fi(list_):
    if type(list_[0]) == list:
        return "not a vector"
    return[[i] for i in list_ ]

In [10]:
def mat_dot(a,b):
    if len(t.tensor_struct(a))>1:
        return t.tensor_contrac(a, b ,[1], [0])
    else:
        return t.tensor_contrac(a, b ,[0], [0])

In [11]:
def eigen_valu1(a):   #using Aberth method can't figure out the best way to initialize
    if len(a) != len(a[0]):
        return "not a square matrix"
    I = [[1 if i ==j else 0 for i in range(len(a)) ]for j in range(len(a)) ]
    def eigen_ma(x):
        return t.list_add(a, t.list_nu(I, -x))
    def de_eipo(x):
        return -1*sum([det(matrix_adj(eigen_ma(x), i,i)) for i in range(len(a))])
    def pp (list_):
        return  [de_eipo(x)/det(eigen_ma(x)) for x in list_]
    def inter(root):
        list_=[[root[i]-j for j in root[:i]+root[i+1:]] for i in range(len(root))]
        list_1=[reduce(lambda x,y: x*y, k) for k in list_]
        return t.list_power(list_1, -1)
    #if len(a)%2 ==0:
        #root = [random.uniform(i*10, i*10 +10) for i in range(len(a)//2)]\
            #+[random.uniform(-j*10-10, -j*10 -20) for j in range(len(a)//2)]
    #else:
        #root = [random.uniform(i*5, i*5 +5) for i in range(len(a)//2)]\
                #+[random.uniform(-j*5-5, -j*5 -10) for j in range(len(a)-len(a)//2)]
    root = [-1,1, Ima(0,1)]
    while True:
        new_root = t.list_sub(root, t.list_power(t.list_sub(pp(root), inter(root)),-1))
        root = new_root
        #print [det(eigen_ma(x)) for x in root]
        #print [det(eigen_ma(x)) for x in root]
        ##print root
        if max([abs(det(eigen_ma(x))) for x in root])<0.0000001:
            break
    return root,max([det(eigen_ma(x)) for x in root])

In [38]:
def eigen_valu(a):   #using newton, one root at a time, good for imaginary root
    if len(a) != len(a[0]):
        return "not a square matrix"
    I = [[1 if i ==j else 0 for i in range(len(a)) ]for j in range(len(a)) ]
    def eig_ma(x):
        return t.list_add(a, t.list_nu(I, -x))
    def de_eipo(x):
        return -1*sum([det(matrix_adj(eig_ma(x), i,i)) for i in range(len(a))])
    def p(root, x):
        if root==[]:
            return det( eig_ma(x))
        else:
            return det( eig_ma(x))/t.list_mu(t.list_shift(root, -x))
    def de_p(root, x):
        if root==[]:
            return de_eipo(x)
        else:
            pl=[det( eig_ma(x)) ]+t.list_power(t.list_shift(root, -x), -1)
            de_pl = [de_eipo(x)]+t.list_power(t.list_shift(root, -x), -2)
            return sum([t.list_mu(pl[:i]+[de_pl[i]]+pl[i+1:]) for i in range(len(pl))] )
    root = []
    while True:
        x = Ima(random.uniform(-10, 10), random.uniform(-10, 10))
        while True:
            x = x- p(root, x)/de_p(root, x)
            if abs( det(eig_ma(x))) < 0.000000001:
                break 
        root.append(x)
        if len(root) == len(a):
            break
    return sorted(root, reverse=True, key = lambda x: x.re)

In [39]:
eigen_valu([[1,2,3,4], [5,6,7,8], [9, 8 ,7, 6], [ 5, 4, 3, 2]])

[20.0+2.2351197039214118e-15i,
 2.122947317820885e-07+2.9300257035089834e-06i,
 -2.9429169481751176e-07-3.0055327077975883e-06i,
 -4.0+6.617444900424222e-24i]

In [14]:
eigen_valu([[1/2, 3**(1/2)/2],[-(3**(1/2)/2), 1/2]])

[0.5000000000035216+0.8660254037867865i,
 0.49999999999749856-0.8660254037887423i]

In [44]:
def eigen_vec(a, ev):
    I = [[1 if i ==j else 0 for i in range(len(a)) ]for j in range(len(a)) ]
    def eig_ma(x):
        return t.list_add(a, t.list_nu(I, -x))
    vec = []
    for v in sorted(Counter(ev).keys(), reverse=True):
        b =eig_ma(v)
        for i in range(len(b)):
            b = matrix_adj(b ,len(b)-1,len(b)-1)
            if abs(det(b)) > 0.000000001:
                c= [j[len(eig_ma(v))-i-1:] for j in eig_ma(v)[:len(eig_ma(v))-1-i] ]  #first slice from top then from bottom
                break                                                                #c has structure [ len(a)-i-1,i+1]
        x_free = [[1 if k==j else 0 for k in range(i+1) ]for j in range(i+1) ]
        x_rem= [mat_dot(matrix_inver(b) , t.list_nu(mat_dot(c, x), -1)) for x in x_free]
        vec.extend([xr+xf for (xr,xf) in zip(x_rem, x_free) ])
    return vec

In [45]:
eigen_vec([[1,2,3,4], [5,6,7,8], [9, 8 ,7, 6], [ 5, 4, 3, 2]],[20, -4, 0,0])

[[0.7560975609756098, 1.926829268292683, 2.1707317073170733, 1],
 [1.0, -2.0, 1, 0],
 [2.0, -3.0, 0, 1],
 [-1.0, -1.0, 1.0, 1]]

In [17]:
ff=mat_dot([[1,2,3,4], [5,6,7,8], [9, 8 ,7, 6], [ 5, 4, 3, 2]], t.transp([[1.0, -2.0, 1, 0],
 [2.0, -3.0, 0, 1],
 [-1.0, -1.0, 1.0, 1],
 [0.7560975609756098, 1.926829268292683, 2.1707317073170733, 1]], [1,0]))

In [18]:
gg =matrix_inver(t.transp([[1.0, -2.0, 1, 0],
 [2.0, -3.0, 0, 1],
 [-1.0, -1.0, 1.0, 1],
 [0.7560975609756098, 1.926829268292683, 2.1707317073170733, 1]], [1,0]))

In [19]:
mat_dot(gg, ff)

[[0.0, 0.0, 0.0, -1.7763568394002505e-15],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, -4.0, -2.6645352591003757e-15],
 [0.0, 0.0, 0.0, 20.000000000000004]]

In [20]:
t.list_nu([1, 2.548387096774194, 2.870967741935484, 1.32258064516129], 20)

[20, 50.96774193548388, 57.41935483870968, 26.4516129032258]

In [21]:
def antisym_fi( tensor, index):
    struct = t.tensor_struct(tensor)
    if len(Counter([struct[ind] for ind in index ] ).keys() ) != 1:
        return "wrong index"
    kd = gk_delta(len(index), struct[index[0]])
    T= t.tensor_contrac(tensor, kd, index, range(len(index)))
    return t.list_nu(T, 1/reduce(lambda x,y: x*y, [i+1 for i in range(len(index)) ])  )

In [22]:
def tol_sym_tensor(lenth, dim):                      #smmetric tensor, we want up index and down index
    struct = [dim for j in range(2*lenth)]           # =1 if the up index is the arrangement 
    bl = [0 for k in range(dim**(2*lenth))]
    def factorial(n):
        return reduce(lambda x,y: x*y, [i+1 for i in range(n) ]) 
    for i in range(len(bl)):
        whole_cord = t.n_to_cord(struct, i)
        u_cord=whole_cord[0:lenth]
        d_cord=whole_cord[lenth:]
        if  Counter(u_cord).keys() != Counter(d_cord).keys(): 
            continue  
        if max(Counter(u_cord).values()) ==1:
            bl[i] = 1
        else:
            factor = Counter(u_cord).values()
            facfac=[factorial(j) for j in factor]
            bl[i] =reduce(lambda x,y: x*y, facfac) 
    return t.list_to_tensor(bl, struct)

In [23]:
def tol_sym_fi( tensor, index):
    struct = t.tensor_struct(tensor)
    if len(Counter([struct[ind] for ind in index ] ).keys() ) != 1:
        return "wrong index"
    st = tol_sym_tensor(len(index), struct[index[0]])
    T= t.tensor_contrac(tensor, st, index, range(len(index)))
    return t.list_nu(T, 1/reduce(lambda x,y: x*y, [i+1 for i in range(len(index)) ])  )

In [24]:
def softmax(list_):
    if type(list_) != list:
        return "must be a list"
    return t.list_nu([ exp(l) for l in  list_], 1/sum([ exp(l) for l in  list_] ) )

In [25]:
def kl_div(q,p):
    if len(q) != len(p):
        return"different lenth"
    inter=[]
    for (qi,pi) in zip(q,p):
        if qi !=0:
            inter.append(qi*log(qi/pi))
    return sum(inter )